# Tensorboard Basics

# Resources

* [Tensorflow Github - tensorboard](https://github.com/tensorflow/tensorboard)

> This README gives an overview of key concepts in TensorBoard, as well as how to interpret the visualizations TensorBoard provides.
> Make sure you have generated summary data in a log directory by creating a summary writer:
> ```
> file_writer = tf.summary.FileWriter('/path/to/logs', sess.graph)
> ```

* [Tensorflow TensorBoard - Visualize your learning](https://jhui.github.io/2017/03/12/TensorBoard-visualize-your-learning/)
* [Aladdinpersson - TensorFlow Tutorial 17 - Complete TensorBoard Guide](https://www.youtube.com/watch?v=k7KfYXXrOj0) ([github](https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/TensorFlow/Basics/tutorial17-tensorboard))
* [Neptune AI - Deep Dive Into TensorBoard: Tutorial With Examples](https://neptune.ai/blog/tensorboard-tutorial)

## Using Tensorboard in Notebook
* [Using TensorBoard in Notebooks](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks)

In [2]:
# Load the TensorBoard notebook extension for Jupyter Notebooks
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


---
# Views

## Metrics

* **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the **Time Series** or **Scalars** dashboards.



## Parameters (Weight & Bias)
* **Distributions** show the distribution of a Tensor over time. This can be useful to visualize **weights and biases** and verify that they are changing in an expected way. Histograms can be found in the **Time Series** or **Histograms** dashboards. Distributions can be found in the **Distributions** dashboard.


## Graph

[Examining the TensorFlow Graph](https://www.tensorflow.org/tensorboard/graphs)

**Graphs** visualize your model’s structure and ensure it matches the intended design. And op-level graph to understand how TensorFlow understands your program.

By default, TensorBoard displays the **op-level graph**. Note that **the graph is inverted**; data flows from bottom to top, so it’s upside down compared to the code. 

## Image

* [Displaying image data in TensorBoard](https://www.tensorflow.org/tensorboard/image_summaries)

> TensorFlow Image Summary API let you log tensors and your images (e.g, Confusion Matrix).
> 1. Create the Keras TensorBoard callback to log basic metrics
> 2. Create a Keras LambdaCallback to log the confusion matrix at the end of every epoch
> 3. Train the model using Model.fit(), making sure to pass both callbacks

```
file_writer_confusion_matrix = tf.summary.create_file_writer(logdir + '/cm')

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)

  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_confusion_matrix.as_default():
    tf.summary.image("epoch_confusion_matrix", cm_image, step=epoch)


# Define the per-epoch callback.
cofusion_matrixm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[cofusion_matrixm_callback],
    validation_data=(test_images, test_labels),
)
```

<img src="image/tensorboard_image.png" align="left" width=700/>

## Hyper Parameters

* [Hyperparameter Tuning with the HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)

> The HParams dashboard help identifying the best experiment or most promising sets of hyperparameters.
> Experiment with three hyperparameters in the model:
> 1. Number of units in the first dense layer
> 2. Dropout rate in the dropout layer
> 3. Optimizer
>  
> ```
> # Load the TensorBoard notebook extension
> %load_ext tensorboard
> 
> import tensorflow as tf
> from tensorboard.plugins.hparams import api as hp
>
> HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
> HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
> HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
> 
> METRIC_ACCURACY = 'accuracy'
> 
> with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
>   hp.hparams_config(
>     hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
>     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
>   )
>
> def train_test_model(hparams):
>   model = tf.keras.models.Sequential([
>     tf.keras.layers.Flatten(),
>     tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
>     tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
>     tf.keras.layers.Dense(10, activation=tf.nn.softmax),
>   ])
>   model.compile(
>       optimizer=hparams[HP_OPTIMIZER],
>       loss='sparse_categorical_crossentropy',
>       metrics=['accuracy'],
>   )
> 
>   model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
>   _, accuracy = model.evaluate(x_test, y_test)
>   return accuracy
>
> def run(run_dir, hparams):
>   with tf.summary.create_file_writer(run_dir).as_default():
>     hp.hparams(hparams)  # record the values used in this trial
>     accuracy = train_test_model(hparams)
>     tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
>
> session_num = 0
> 
> for num_units in HP_NUM_UNITS.domain.values:
>   for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
>     for optimizer in HP_OPTIMIZER.domain.values:
>       hparams = {
>           HP_NUM_UNITS: num_units,
>           HP_DROPOUT: dropout_rate,
>           HP_OPTIMIZER: optimizer,
>       }
>       run_name = "run-%d" % session_num
>       print('--- Starting trial: %s' % run_name)
>       print({h.name: hparams[h] for h in hparams})
>       run('logs/hparam_tuning/' + run_name, hparams)
>       session_num += 1
> ```

The HParams dashboard has three different views, with various useful information:

1. The Table View lists the runs, their hyperparameters, and their metrics.
2. The Parallel Coordinates View shows each run as a line going through an axis for each hyperparemeter and metric. Click and drag the mouse on any axis to mark a region which will highlight only the runs that pass through it. This can be useful for identifying which groups of hyperparameters are most important. The axes themselves can be re-ordered by dragging them.
3. The Scatter Plot View shows plots comparing each hyperparameter/metric with each metric. This can help identify correlations. Click and drag to select a region in a specific plot and highlight those sessions across the other plots.
A table row, a parallel coordinates line, and a scatter plot market can be clicked to see a plot of the metrics as a function of training steps for that session (although in this tutorial only one step is used for each run).

```(number of units, dropout rate, optimizer, accuracy) = (128, 0.2, adam, 0.9783)``` gives the best accuracy.

<img src="image/tensorboard_hdparam.png" align="left" width=700/>

## Embedding Projection

> Embedding Projector help visualizing, examining, and understanding your embedding layers. In order to load the data into Tensorboard, we need to save a training checkpoint to that directory, along with metadata that allows for visualization of a specific layer of interest in the model.
>
> ```
> # Create an embedding layer.
> embedding_dim = 16
> embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
> 
> # Configure the embedding layer as part of a keras model.
> model = tf.keras.Sequential(
>     [
>         embedding, # The embedding layer should be the first layer in a model.
>         tf.keras.layers.GlobalAveragePooling1D(),
>         tf.keras.layers.Dense(16, activation="relu"),
>         tf.keras.layers.Dense(1),
>     ]
> )
> 
> # Save Labels separately on a line-by-line manner.
> with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
>   for subwords in encoder.subwords:
>     f.write("{}\n".format(subwords))
>   # Fill in the rest of the labels with "unknown".
>   for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
>     f.write("unknown #{}\n".format(unknown))
> 
> # Save the weights we want to analyze as a variable. Note that the first
> # value represents any unknown word, which is not in the metadata, here
> # we will remove this value.
> weights = tf.Variable(model.layers[0].get_weights()[0][1:])
> 
> # Create a checkpoint from embedding, the filename and key are the
> # name of the tensor.
> checkpoint = tf.train.Checkpoint(embedding=weights)
> checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))
> 
> # Set up config.
> config = projector.ProjectorConfig()
> embedding = config.embeddings.add()
> 
> # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
> embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
> embedding.metadata_path = 'metadata.tsv'
> projector.visualize_embeddings(log_dir, config)
> ```

<img src="image/tensorboard_projector.png" align="left" width=700/>

## Debugger V2 - Nan/Inf Detection

* [Debugging Numerical Issues](https://www.tensorflow.org/tensorboard/debugger_v2)

> TensorBoard 2.3+ (together with TensorFlow 2.3+) provides a specialized dashboard called Debugger to detect errors involving NaNs.
> 